In [2]:
!pip install mxnet

     |████████████████████████████████| 46.9 MB 42 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [14]:
import mxnet as mx
import numpy as np
import matplotlib.pyplot as plt
from mxnet import nd,autograd,gluon
from mxnet.gluon import nn

In [15]:
td=mx.gluon.data.vision.MNIST(train=True)
vd=mx.gluon.data.vision.MNIST(train=False)

In [16]:
def f_transform(data):
  return nd.moveaxis(data,2,0).astype('float32')/255

In [17]:
train_data=td.transform_first(f_transform)
val_data=vd.transform_first(f_transform)

In [18]:
batch_size=50
tL=mx.gluon.data.DataLoader(train_data,shuffle=True,batch_size=batch_size)

In [19]:
vL=mx.gluon.data.DataLoader(val_data,shuffle=True,batch_size=batch_size)

In [20]:
model=nn.Sequential()
with model.name_scope():
  model.add(
  nn.Flatten(),
  nn.Dense(128,activation='relu'),
  nn.Dense(64,activation='relu'),
  nn.Dense(32,activation='relu'),
  nn.Dense(10,activation=None)
  )

In [21]:
ctx=mx.cpu(0)

In [22]:
model.initialize(mx.init.Xavier(),ctx)

In [23]:
trainer=gluon.Trainer(
    params=model.collect_params(),
    optimizer='sgd',
    optimizer_params={'learning_rate':0.01},
)

In [24]:
metric=mx.metric.Accuracy()
loss_function=gluon.loss.SoftmaxCrossEntropyLoss()

In [25]:
num_epochs=5

for epoch in range(num_epochs):
  for X,y in tL:
    X= X.as_in_context(ctx)
    y= y.as_in_context(ctx)

    with autograd.record():
      y_hat=model(X)
      loss=loss_function(y_hat,y)

    loss.backward()
    metric.update(y,y_hat)
    trainer.step(batch_size=X.shape[0])

  name,acc=metric.get()
  print(acc)
  metric.reset()

0.7660166666666667
0.9078666666666667
0.9251333333333334
0.9360666666666667
0.9438666666666666


In [26]:
metric=mx.metric.Accuracy()
for Xv,yv in vL:
  Xv=Xv.as_in_context(ctx)
  yv=yv.as_in_context(ctx)
  metric.update(yv,model(Xv))
name,acc=metric.get()
print(acc)

0.9469


In [27]:
model = nn.Sequential()
with model.name_scope():
  model.add(
      nn.Conv2D(channels=32,kernel_size=(5,5),activation='relu'),
      nn.MaxPool2D(pool_size=(2,2),strides=(2,2)),
      nn.Dropout(0.3),
      nn.Conv2D(channels=64,kernel_size=(5,5),activation='relu'),
      nn.MaxPool2D(pool_size=(2,2),strides=(2,2)),
      nn.Flatten(),
      nn.Dense(128,activation='relu'),
      nn.Dense(10,activation=None),
  

  )

In [28]:
model.initialize(mx.init.Xavier())

In [29]:
trainer=gluon.Trainer(
    params=model.collect_params(),
    optimizer= 'sgd',
    optimizer_params={'learning_rate':0.02}
)

In [30]:
metric=mx.metric.Accuracy()
loss_function=mx.gluon.loss.SoftmaxCrossEntropyLoss()

In [31]:
ctx=mx.cpu(0)

epochs=5
for ep in range(epochs):
  for X,y in tL:
    X=X.as_in_context(ctx)
    y=y.as_in_context(ctx)

    with autograd.record():
      y_hat=model(X)
      loss=loss_function(y_hat,y)
    loss.backward()
    metric.update(y,y_hat)
    trainer.step(batch_size=X.shape[0])
  name,acc=metric.get()
  print(acc)
  metric.reset()

0.8664666666666667
0.962
0.9718
0.9777
0.9813


In [32]:
metric=mx.metric.Accuracy()
for Xv,yv in vL:
  Xv=Xv.as_in_context(ctx)
  yv=yv.as_in_context(ctx)
  metric.update(yv,model(Xv))
name,acc=metric.get()
print(acc)

0.9863
